In [1]:
from data import Data
from dimension_reduction import PCADimensionReduction
from autoencoder import Autoencoder
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import optimizers

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_filepath = "data/by_type/t1/counts_ctc_simulated_123_5k_t1.tsv"
true_results_filepath = "data/by_type/t1/ids_ctc_simulated_123_5k_t1.tsv"
train_indices_filepath = "data/by_type/t1/train_indices.npy"
test_indices_filepath = "data/by_type/t1/test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4
FILTER_BY_ZERO_THRESHOLD = 0.97

In [3]:
data_object = Data(data_filepath, true_results_filepath)

train_data, test_data, train_true_results, test_true_results = data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)

In [4]:
healthy_train_data, cancer_train_data, healthy_test_data, cancer_test_data = data_object.load_train_test_healthy_cancer_data()


In [5]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(50, input_dim=input_shape, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(input_shape))
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

## Autoencoder

### Regular data

In [6]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

2022-11-27 23:06:20.427588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:06:20.482678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:06:20.483100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:06:20.484019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/100
132/190 [===================>..........] - ETA: 0s - loss: 0.1818

2022-11-27 23:06:22.540848: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


190/190 [==============================] - 1s 1ms/step - loss: 0.1643
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1237
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1189
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1173
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1161
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1154
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1150
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1146
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1142
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1138
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1134
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1131
Epoch 13/100
190/190 [===

In [7]:
autoencoder_object = Autoencoder(autoencoder, healthy_train_data, healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 665us/step
Healthy train RMSE: 0.32789359175315536
Healthy test RMSE: 0.3341379708915169
Cancer train RMSE: 0.5131853528166589
Cancer test RMSE: 0.5140926171461522
Threshold: 0.4205394722849071


In [8]:
autoencoder_object.display_regular_metrics(train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(test_data, test_true_results, "Test")

97/97 [==============================] - 0s 665us/step
Train balanced accuracy: 0.9888121092464627
Train f1 score: 0.5072463768115942
Train precision: 0.33980582524271846
Train recall: 1.0
42/42 [==============================] - 0s 688us/step
Test balanced accuracy: 0.9842670759785112
Test f1 score: 0.4225352112676056
Test precision: 0.26785714285714285
Test recall: 1.0


### Scaled data

In [6]:
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data = data_object.get_scaled_healthy_train_test_cancer_data()
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 2s 3ms/step - loss: 0.9311
Epoch 2/100
190/190 [==============================] - 1s 3ms/step - loss: 0.9067
Epoch 3/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8985
Epoch 4/100
190/190 [==============================] - 0s 2ms/step - loss: 0.8939
Epoch 5/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8907
Epoch 6/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8877
Epoch 7/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8846
Epoch 8/100
190/190 [==============================] - 0s 2ms/step - loss: 0.8820
Epoch 9/100
190/190 [==============================] - 0s 3ms/step - loss: 0.8800
Epoch 10/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8786
Epoch 11/100
190/190 [==============================] - 0s 3ms/step - loss: 0.8756
Epoch 12/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8741
Epoch 13/100


In [7]:
autoencoder_object = Autoencoder(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data)
autoencoder_object.display_average_rmse()

Healthy train RMSE: 0.8960805442161728
Healthy test RMSE: 0.9417005353342102
Cancer train RMSE: 7.830166850303598
Cancer test RMSE: 7.856618203151696
Threshold: 4.363123697259885


In [8]:
scaled_train_data = data_object.scaled_data_by_healthy_train(train_data)
scaled_test_data = data_object.scaled_data_by_healthy_train(test_data)

autoencoder_object.display_regular_metrics(scaled_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(scaled_test_data, test_true_results, "Test")

Train balanced accuracy: 0.9857142857142858
Train f1 score: 0.9855072463768115
Train precision: 1.0
Train recall: 0.9714285714285714
Test balanced accuracy: 0.9984650805832693
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0


### Cut by max data

In [12]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 914us/step - loss: 1.5323
Epoch 2/100
190/190 [==============================] - 0s 911us/step - loss: 0.5419
Epoch 3/100
190/190 [==============================] - 0s 907us/step - loss: 0.4931
Epoch 4/100
190/190 [==============================] - 0s 913us/step - loss: 0.4537
Epoch 5/100
190/190 [==============================] - 0s 912us/step - loss: 0.4360
Epoch 6/100
190/190 [==============================] - 0s 912us/step - loss: 0.4249
Epoch 7/100
190/190 [==============================] - 0s 908us/step - loss: 0.4147
Epoch 8/100
190/190 [==============================] - 0s 909us/step - loss: 0.4072
Epoch 9/100
190/190 [==============================] - 0s 911us/step - loss: 0.4020
Epoch 10/100
190/190 [==============================] - 0s 911us/step - loss: 0.3969
Epoch 11/100
190/190 [==============================] - 0s 912us/step - loss: 0.3929
Epoch 12/100
190/190 [==============================] - 0s 915us/step - lo

In [13]:
autoencoder_object = Autoencoder(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 546us/step
Healthy train RMSE: 0.5676580992712108
Healthy test RMSE: 0.5810853139664297
Cancer train RMSE: 0.8203384152495308
Cancer test RMSE: 0.8181373436151395
Threshold: 0.6939982572603708


In [14]:
cut_by_max_train_data = data_object.cut_by_max_healthy_data(train_data, CUT_BY_MAX_THRESHOLD)
cut_by_max_test_data = data_object.cut_by_max_healthy_data(test_data, CUT_BY_MAX_THRESHOLD)

autoencoder_object.display_regular_metrics(cut_by_max_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(cut_by_max_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 551us/step
Train balanced accuracy: 0.8637474733229915
Train f1 score: 0.130718954248366
Train precision: 0.07075471698113207
Train recall: 0.8571428571428571
42/42 [==============================] - 0s 556us/step
Test balanced accuracy: 0.8228702993092862
Test f1 score: 0.10526315789473684
Test precision: 0.056338028169014086
Test recall: 0.8


### Filtered by zero values for scaled data

In [9]:
filtered_by_zero_values_train_data, filtered_by_zero_values_test_data, filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_test_data, cancer_train_data, cancer_test_data = data_object.get_filtered_by_zero_data(scaled_train_data, scaled_test_data, FILTER_BY_ZERO_THRESHOLD)

Number of deleted columns: 93


In [10]:
autoencoder = create_autoencoder(filtered_by_zero_healthy_train_data.shape[1])
autoencoder.fit(filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 1s 3ms/step - loss: 0.9799
Epoch 2/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9550
Epoch 3/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9433
Epoch 4/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9368
Epoch 5/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9326
Epoch 6/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9295
Epoch 7/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9268
Epoch 8/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9237
Epoch 9/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9210
Epoch 10/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9197
Epoch 11/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9180
Epoch 12/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9160
Epoch 13/100


In [11]:
autoencoder_object = Autoencoder(autoencoder, filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

Healthy train RMSE: 0.9182870869227074
Healthy test RMSE: 0.9634428310716991
Cancer train RMSE: 8.038714351251496
Cancer test RMSE: 8.021397837603828
Threshold: 4.478500719087101


In [12]:
autoencoder_object.display_regular_metrics(filtered_by_zero_values_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(filtered_by_zero_values_test_data, test_true_results, "Test")

Train balanced accuracy: 0.9857142857142858
Train f1 score: 0.9855072463768115
Train precision: 1.0
Train recall: 0.9714285714285714
Test balanced accuracy: 0.9984650805832693
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0
